In [ ]:
from sctoolbox.utils.jupyter import bgcolor, _compare_version

# change the background of input cells
bgcolor("PowderBlue", select=[2, 4, 6, 8, 10, 12])

nb_name = "0A2_ligand_receptor_differences.ipynb"

_compare_version(nb_name)

# 0A2 - Receptor-Ligand Differential Analysis

## 1 - Description


This notebook is designed to be run after 0A1_ligand_receptor.ipynb.

Cell-cell communication plays a crucial role in tissue function, development, and disease progression. When experimental conditions change (e.g., treatment vs. control) or as time progresses, these communication patterns can evolve significantly. This notebook helps identify and visualize these changes in receptor-ligand interactions across:

1. Different experimental conditions (e.g., treatment vs. control)
2. Different timepoints in a time series experiment

By identifying which cell-cell interactions change between conditions or over time, it is possible to:

- Understand mechanisms driving different cellular responses
- Identify key signaling pathways that respond to experimental interventions
- Track the evolution of intercellular communication during processes like development, differentiation, or disease progression

The differential analysis computes quantile-ranked differences in receptor-ligand interaction strength and visualizes these differences through network graphs that highlight the most significantly changing interactions.

____

## 2 - Setup

In [ ]:
import sctoolbox.tools.receptor_ligand as rl
import sctoolbox.utils as utils
import pandas as pd
import matplotlib.pyplot as plt
from sctoolbox import settings

settings.settings_from_config("config.yaml", key="0A2")

____

## 3 - Load adata

Load an annotated AnnData object with cluster annotations and ideally the basic receptor-ligand analysis already performed using the 0A1 notebook. This object should contain different experimental conditions or timepoints to enable differential analysis.

# <h1><center>⬐ Fill in input data here ⬎</center></h1>

In [ ]:
# Input filename
last_notebook_adata = "anndata_0A1.h5ad"

In [ ]:
adata = utils.adata.load_h5ad(last_notebook_adata)

with pd.option_context("display.max.rows", 5, "display.max.columns", None):
    display(adata)
    display(adata.obs)
    display(adata.var)

____

## 4 - Difference Analysis

The difference analysis identifies receptor-ligand interactions that change between different conditions or across timepoints. This is crucial for understanding how cell-cell communication adapts to perturbations or evolves during biological processes.
The analysis involves:

Calculating receptor-ligand interaction scores for each condition or timepoint separately
Computing quantile-ranked differences to identify which interactions change most dramatically
Visualizing the changing interactions as network graphs

The notebook covers two main types of analysis:

- Pairwise Analysis: Compare interactions between specific conditions (e.g., treatment vs. control)
- Progressive Analysis: Track how interactions evolve across an ordered series (typically timepoints)

### 4.1 Pairwise Analysis Between Conditions

Pairwise analysis compares receptor-ligand interactions between different experimental conditions to identify which interactions become stronger or weaker. This helps discover signaling pathways that respond to experimental settings.

How to Interpret the Network Plots:

1. Nodes represent cell type-specific receptor or ligand genes. The shape and color of each node corresponds to a cell type (see the legend on the right side of the plot).
2. Edges (arrows) represent receptor-ligand interactions, pointing from ligand to receptor. The color intensity of the edge indicates the magnitude of change in interaction strength between conditions.
3. Hub Networks: Genes with many connections (≥ hub_threshold) are displayed separately as "hubs". These represent key players that show changed interaction patterns across multiple signaling pathways.
4. Quantile rank differences: The colorbar at the bottom indicates the magnitude and direction of change in interaction strength. In positive difference plots (red), interactions are stronger in condition B compared to condition A. In negative difference plots (blue), interactions are stronger in condition A.
5. Direction: The arrows always point from ligand to receptor, showing the directionality of the signaling.

<h1><center>⬐ Fill in input data here ⬎</center></h1>

In [ ]:
# Main parameters for condition comparison
condition_columns = ['condition', 'timepoint']  # Columns in adata.obs for hierarchical comparison
                                               # First column is primary comparison dimension
cluster_column = "celltype"   # Column containing cell type/cluster labels
gene_column = None           # Column in adata.var with gene symbols (None to use index)
normalize = None             # Normalize cluster sizes (None uses max cluster size)

save_diff = True  # Save difference tables to disk

# Optional filtering to focus on specific conditions, clusters, or genes
condition_filters = {}       # Restrict to specific values in condition columns
                             # Example: {"condition": ["control"], "timepoint": ["tp1", "tp3"]}
selected_clusters = None     # Restrict to specific clusters (list of names)
selected_genes = None        # Restrict to specific genes (list of names)

# Expression and interaction filtering - it is recommended to refreine from filtering before difference calculation
min_perc = None                 # Minimum % of cells expressing each gene (0-100)
interaction_perc = None       # Percentile threshold for interactions (0-100)

# Data layer (advanced)
# decide which processing state of the data should be used for computation
# available layers can be seen above, e.g. "norm" to use normalized data
# it is recommended to use raw or normalized data for statistical testing
layer = "norm"

In [ ]:
# Calculate differences between conditions
rl.calculate_condition_differences(
    adata=adata,
    condition_columns=condition_columns,
    cluster_column=cluster_column,
    condition_filters=condition_filters,
    cluster_filter=selected_clusters,
    gene_filter=selected_genes,
    gene_column=gene_column,
    normalize=normalize,
    min_perc=min_perc,
    interaction_perc=interaction_perc,
    inplace=True,
    overwrite=True,
    save_diff=save_diff,
    layer=layer
)

<h1><center>⬐ Fill in input data here ⬎</center></h1>

In [ ]:
# Visualization parameters
n_top = 100                 # Number of top differential interactions to show
figsize = (24, 18)          # Figure size in inches
dpi = 300                   # Resolution of the figure
split_by_direction = True   # Create separate plots for up/down-regulated interactions
hub_threshold = 4           # Threshold that defines a hub
save_prefix = "condition_comparison_networks"  # Prefix for saved figures
save_format = "pdf"         # Format of saved figures

In [ ]:
# Visualize differences between conditions as networks
figures = rl.plot_all_condition_differences(
    adata=adata,
    n_top=n_top,
    hub_threshold=hub_threshold,
    figsize=figsize,
    dpi=dpi,
    split_by_direction=split_by_direction,
    save = (save_prefix, save_format)
)

____

### 4.2 - Progressive Analysis Across Timepoints

Progressive analysis tracks how receptor-ligand interactions evolve over a series of ordered points. This helps understand the dynamics of cell-cell communication during processes like disease progression.

<h1><center>⬐ Fill in input data here ⬎</center></h1>

In [ ]:
# Main parameters for condition comparison
condition_columns = ['timepoint', 'condition']  # First column must be the time dimension,
                                                # i.e. the interaction differences will be calculated
                                                # between the values of the first condition column in the list
                                                # with respect to all combination of the subsequent conditions.

                                                
cluster_column = "celltype"   # Column containing cell type/cluster labels

time_column = 'timepoint' # Column containing the timepoints
time_order = ['tp1', 'tp2', 'tp3'] # Define the ordering of the timepoints

gene_column = None           # Column in adata.var with gene symbols (None to use index)
normalize = None             # Normalize cluster sizes (None uses max cluster size)

save_diff = True  # Save difference tables to disk

# Optional filtering to focus on specific conditions, clusters, or genes
condition_filters = {}       # Restrict to specific values in condition columns
                             # Example: {"condition": ["control"], "timepoint": ["tp1", "tp3"]}
selected_clusters = None     # Restrict to specific clusters (list of names)
selected_genes = None        # Restrict to specific genes (list of names)

# Expression and interaction filtering - it is recommended to refreine from filtering before difference calculation
min_perc = None                 # Minimum % of cells expressing each gene (0-100)
interaction_perc = None       # Percentile threshold for interactions (0-100)

inplace = True
overwrite = True

In [ ]:
rl.calculate_condition_differences(
    adata=adata,
    condition_columns=condition_columns,
    cluster_column=cluster_column,
    time_column=time_column,
    time_order=time_order,
    condition_filters=condition_filters,
    min_perc=min_perc,
    inplace=inplace,
    overwrite=overwrite,
    layer=layer
)

<h1><center>⬐ Fill in input data here ⬎</center></h1>

In [ ]:
# Visualization parameters
n_top = 100                 # Number of top differential interactions to show
figsize = (24, 18)          # Figure size in inches
dpi = 300                   # Resolution of the figure
split_by_direction = True   # Create separate plots for up/down-regulated interactions
hub_threshold = 4            # Threshold that defines a hub
save_prefix = "timecourse_treatment"  # Prefix for saved figures
save_format = "pdf"
n_cols = 4 # Number of subplot columns

In [ ]:
# Visualize the time-based differences
figures = rl.plot_all_condition_differences(
    adata=adata,
    n_top=n_top, 
    split_by_direction=True,
    hub_threshold=hub_threshold,
    color_palette='tab20',
    n_cols=n_cols,                     
    show=True,
    save=(save_prefix, save_format)
)

____

## 4.3 Plot specfic interactions over time

If specific interactions of interest are detected in the network graphs, it is possible to plot the evolution of the receptor and ligand expression levels over different timepoints for the interactions of interest.

How to Interpret the Bar Plots:

These bar plots show the raw expression levels of specific receptor-ligand pairs across timepoints, providing a more detailed view of how individual interactions change over time.

1. Bar Colors: Blue bars represent receptor expression, orange bars represent ligand expression. (Default settings.)
2. X-axis: Shows the different timepoints in chronological order.
3. Y-axis: Shows the (mean) expression level of the gene in the specified cell type.

<h1><center>⬐ Fill in input data here ⬎</center></h1>

In [ ]:
# Define interactions of interest
interactions = [
    # (receptor_gene, receptor_cluster, ligand_gene, ligand_cluster)
    ('CD44', 'LC', 'TIMP3', 'FB')
]

cluster_column = "celltype"   # Column containing cell type/cluster labels
time_column = 'timepoint' # Column containing the timepoints
time_order = ['tp1', 'tp2', 'tp3'] # Define the ordering of the timepoints

gene_column = None 
figsize = None # Figure dimensions in inches.
dpi = 300 # Figure resolution.
save = None # Output filename.
title = None # Overall figure title.
n_cols = 2 # Number of columns in the grid layout.
receptor_color = None # Color for receptor bars.
                      # If None, uses the first color from seaborn's default palette.
ligand_color = None   # Color for ligand bars.
                      # If None, uses the second color from seaborn's default palette.


In [ ]:
figure = rl.plot_interaction_timeline(
    adata=adata,
    interactions=interactions,
    timepoint_column=time_column,
    cluster_column=cluster_column,
    time_order=time_order,
    layer=layer,
    save=save
)

## 5 - Saving adata

Save the AnnData object with all the differential receptor-ligand analysis results for future reference or further analysis.

In [ ]:
utils.adata.save_h5ad(adata, "anndata_0A2.h5ad")

In [ ]:
settings.close_logfile()